## Set up Environment

In [ ]:
# Install the necessary packages
!pip3 install numpy
!pip3 install tensorflow
!pip3 install keras
!pip3 install scikit-learn
!pip3 install imutils
!pip3 install matplotlib

In [99]:
# Import the necessary packages needed for model training
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

## Load the Dataset

In [65]:
# Set all constants
crying_dataset_path = '/home/srinivassriram/Desktop/Rescue_Pi_Code/Rescue_Pi_Code/Dataset2/Crying/'
crying_image_paths = list(paths.list_images(crying_dataset_path))
fighting_dataset_path = '/home/srinivassriram/Desktop/Rescue_Pi_Code/Rescue_Pi_Code/Dataset2/Fighting/'
fighting_image_paths = list(paths.list_images(fighting_dataset_path))
normal_dataset_path = '/home/srinivassriram/Desktop/Rescue_Pi_Code/Rescue_Pi_Code/Dataset2/Crying/'
normal_image_paths = list(paths.list_images(normal_dataset_path))
data_list = np.array([])
labels_list = np.array([])

In [66]:
# loop over the image paths for crying
for image_path in crying_image_paths:
    label = 'Crying'
    
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    
    np.append(data_list, image)
    np.append(labels_list, label)

In [67]:
# loop over the image paths for fighting
for image_path in fighting_image_paths:
    label = 'Fighting'
    
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    
    np.append(data_list, image)
    np.append(labels_list, label)

In [68]:
# loop over the image paths for normal behavior
for image_path in normal_image_paths:
    label = 'Normal'
    
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    
    np.append(data_list, image)
    np.append(labels_list, label)

In [69]:
print(type(data_list))
print(type(labels_list))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [70]:
# Encode the labels
lb = LabelBinarizer()
labels_array = lb.fit_transform(labels_array)

In [71]:
# Split the dataset into training and testing datasets
(trainX, testX, trainY, testY) = train_test_split(data_array, labels_array, test_size=0.20, stratify=labels_array, random_state=42)

In [72]:
# Construct the image data generator that is needed for data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    brightness_range=[0.2,2.0],
    fill_mode="nearest")

## Build and Train the Model

In [89]:
# Set all constants
INIT_LR = 1e-3
EPOCHS = 100
BS = 16

In [115]:
# Load the resnet network
baseModel = ResNet50(weights="imagenet", include_top=True,
                        input_tensor=Input(shape=(224, 224, 3)))

In [116]:
# Construct the model head
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

ValueError: Input 0 of layer average_pooling2d_10 is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: [None, 1000]

In [117]:
# Fine tune the network with the model we will be training with
model = Model(inputs=baseModel.input, outputs=headModel)

In [118]:
# Freeze all layers in the base model during the first training process
for layer in baseModel.layers:
    layer.trainable = False

In [119]:
# Compile our model
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
              metrics=["accuracy"])

In [120]:
# Train the head of the network
H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

Epoch 1/100


InvalidArgumentError:  Incompatible shapes: [16,3] vs. [16,1000]
	 [[node binary_crossentropy/mul (defined at <ipython-input-120-f446727cc25d>:2) ]] [Op:__inference_train_function_109533]

Function call stack:
train_function
